In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext

from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
#!pip install spacy --upgrade

Since in this assignment, we are only working on English, spacy for english only is needed.

In [ ]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_n(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)] #[::-1]

In [ ]:
QUE1 = Field(tokenize = tokenize_n,
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

QUE2 = Field(tokenize = tokenize_n, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
fields = [('que1', QUE1),('que2',QUE2)]

In [ ]:
from  torchtext.legacy import data
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Only those question pairs have to be selected for which duplicate is true. 

In [ ]:
df=(pd.read_csv('/content/drive/MyDrive/dataset/quora_duplicate_questions.tsv',sep='\t',
               header=0,lineterminator='\n',dtype=pd.StringDtype(),encoding = "ISO-8859-1",
               names= ['id','qid1','qid2','question1','question2','is_duplicate'],
               usecols = ['question1','question2','is_duplicate']))[lambda x: x['is_duplicate'] =='1']

## Overview of the Question Pair Dataset from Quora

The file "quora_duplicate_questions.tsv" contains the questions-question pairs. The first line of the file contains 
column names for the tab-separated data fields in the file, which are:

id qid1 qid2 question1 question2 is_duplicate

Since we are only focussing on the text part of the dataset, which is question1 and question2, I am only picking up these two columns along with is_duplicate to select only those rows for which is_duplicate is true.

So here is number of samples which get selected.

In [ ]:
df.shape

(149263, 3)

Some idea of what the data looks like

In [ ]:
df.head()

,question1,question2,is_duplicate
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,How can I be a good geologist?,What should I do to be a great geologist?,1
11,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,What was your first sexual experience like?,What was your first sexual experience?,1


The first column values which are treated as index now do not have a running number. So needed to reset the index values.

In [ ]:
df=df.reset_index(drop=True)

Another glimpse of the final data with correct indexing

In [ ]:
df.head()

,question1,question2,is_duplicate
0,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
1,How can I be a good geologist?,What should I do to be a great geologist?,1
2,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
3,What can make Physics easy to learn?,How can you make physics easy to learn?,1
4,What was your first sexual experience like?,What was your first sexual experience?,1


Lets first create the DataFrame

In [ ]:
example = [torchtext.legacy.data.Example.fromlist([str(df.question1[i]),str(df.question2[i])], fields) for i in range(df.shape[0])]

Now create the dataset


In [ ]:
from torchtext.legacy import data
QQDataset=data.Dataset(example,fields)

Finally, we can split into training  and test sets by using the split() method:

In [ ]:
(train_set, test_set) = QQDataset.split(split_ratio=[0.70, 0.30], random_state=random.seed(SEED))

In [ ]:
print(f"Number of training examples: {len(train_set.examples)}")
print(f"Number of testing examples: {len(test_set.examples)}")

Number of training examples: 104484
Number of testing examples: 44779


In [ ]:
QUE1.build_vocab(train_set, min_freq = 2)
QUE2.build_vocab(train_set, min_freq = 2)

In [ ]:
print(f"Unique tokens in source  vocabulary: {len(QUE1.vocab)}")
print(f"Unique tokens in target vocabulary: {len(QUE2.vocab)}")

Unique tokens in source  vocabulary: 14579
Unique tokens in target vocabulary: 14533


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator,  test_iterator = BucketIterator.splits(
    (train_set, test_set), sort_key = lambda x: len(x.que1),
    sort_within_batch=True,
    batch_size = BATCH_SIZE, 
    device = device)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = len(QUE1.vocab)
OUTPUT_DIM = len(QUE2.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(14579, 256)
    (lstm): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(14533, 256)
    (lstm): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=14533, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 22,264,517 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = QUE2.vocab.stoi[QUE2.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

My Trials to understand the shape of ouputs etc.Pls Ignore next two cells

In [ ]:
b=next(iter(train_iterator))
src,trg=b.que1,b.que2

In [ ]:
o=model(src,trg)
ou_dim = o.shape[-1]
output = o[1:].view(-1, ou_dim)
tr = trg[1:].view(-1)
optimizer.zero_grad()
loss = criterion(output, tr)
loss.backward()
clip=1      
torch.nn.utils.clip_grad_norm_(model.parameters(), clip)      
optimizer.step()

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.que1
        trg = batch.que2
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.que1
            trg = batch.que2

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
#The name VALID_LOSS actually indicated TEST_LOSS. There was no validation data. The model was evaluated on Test Data.
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 8s
	Train Loss: 4.845 | Train PPL: 127.071
	 Val. Loss: 4.719 |  Val. PPL: 112.104
Epoch: 02 | Time: 4m 8s
	Train Loss: 3.857 | Train PPL:  47.319
	 Val. Loss: 4.208 |  Val. PPL:  67.208
Epoch: 03 | Time: 4m 7s
	Train Loss: 3.325 | Train PPL:  27.790
	 Val. Loss: 3.911 |  Val. PPL:  49.967
Epoch: 04 | Time: 4m 8s
	Train Loss: 3.001 | Train PPL:  20.096
	 Val. Loss: 3.751 |  Val. PPL:  42.558
Epoch: 05 | Time: 4m 9s
	Train Loss: 2.760 | Train PPL:  15.802
	 Val. Loss: 3.675 |  Val. PPL:  39.431
Epoch: 06 | Time: 4m 8s
	Train Loss: 2.567 | Train PPL:  13.033
	 Val. Loss: 3.651 |  Val. PPL:  38.509
Epoch: 07 | Time: 4m 7s
	Train Loss: 2.427 | Train PPL:  11.321
	 Val. Loss: 3.560 |  Val. PPL:  35.156
Epoch: 08 | Time: 4m 6s
	Train Loss: 2.315 | Train PPL:  10.123
	 Val. Loss: 3.599 |  Val. PPL:  36.552
Epoch: 09 | Time: 4m 8s
	Train Loss: 2.221 | Train PPL:   9.213
	 Val. Loss: 3.561 |  Val. PPL:  35.192
Epoch: 10 | Time: 4m 7s
	Train Loss: 2.142 | Train PPL:   8.518


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.560 | Test PPL:  35.156 |
